In [ ]:
# Import the neccesary libraries needed
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
from numpy import insert
import os
import time

# keras tuner for hyperparameter tuning
import keras_tuner as kt

# tensorflow Libraries
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM, Conv2D, MaxPooling2D, Flatten, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import sparse_categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [ ]:
#[10 10 21]
#[1920 1080 3]
#[255 0 0]

In [ ]:
print(tf.test.gpu_device_name())
print(tf.config.experimental.list_physical_devices(device_type='GPU'))

Import the CSV Training and Testing Data Sets

In [ ]:
# Import the csv's (IMPORTANT: You need at least 16 Gb of RAM to proceed)
#train=pd.read_csv(r'C:\Users\Daniel\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\PortTraining_1404_002_FINAL.csv')
#test=pd.read_csv(r'C:\Users\Daniel\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\StbdTesting_1404_002_FINAL.csv')
train = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\PortTraining_1404_002_FINAL_startingAt24.csv')
test = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\StbdTesting_1404_002_FINAL.csv')

In [ ]:
train

In [ ]:
test

Split up the Features and Labels from both the Training and Testing Datasets

In [ ]:
# Split up the features and labels from both the training and testing datasets
x_train=train.iloc[:,4:24]
#x_train['TWTT']=train.iloc[:,28]  I feel as though the TWTT has NO impact on the DOA so we should NOT include it in training
x_train['Amplitude']=train.iloc[:,29]


x_test=test.iloc[:,4:24]
#x_test['TWTT']=test.iloc[:,28] I feel as though the TWTT has NO impact on the DOA so we should NOT include it in training
x_test['Amplitude']=test.iloc[:,29]

y_train=train.iloc[:,26]

y_test=test.iloc[:,26]

In [ ]:
y_test

In [ ]:
x_train

Scale the Features from -1 to 1 so that Training is Easier for the Model

In [ ]:
# Scale the Features and Labels from [-1,1]
s1=MinMaxScaler(feature_range=(0,1))
#s1=MinMaxScaler(feature_range=(-1,1))
x_train_scale=s1.fit_transform(x_train)

#s2=MinMaxScaler(feature_range=(-1,1))
s2=MinMaxScaler(feature_range=(0,1))
x_test_scale=s2.fit_transform(x_test)
x_test_scale=x_test_scale

# Scale DOA's from (0 to 1)
s3=MinMaxScaler(feature_range=(0,1))
#y_train_scale = s3.fit_transform(train[['DOA']])
y_train_scale= y_train.to_numpy()

s4=MinMaxScaler(feature_range=(0,1))
#y_test_scale = s4.fit_transform(test[['DOA']])
y_test_scale= y_test.to_numpy()

print(len(x_train_scale))
print(len(y_train_scale))

In [ ]:
y_train_scale

In [ ]:
Test_DOA_df= pd.DataFrame(y_test_scale, columns=['TEST DOAs'])
Test_DOA_df


In [ ]:
Test_DOA_df.describe()

In [ ]:
df= pd.DataFrame(x_train_scale, columns= ['I1', 'Q1', 'I2', 'Q2', 'I3', 'Q3', 'I4', 'Q4', 'I5', 'Q5', 'I6', 'Q6', 'I7', 'Q7', 'I8', 'Q8', 'I9', 'Q9', 'I10', 'Q10', 'Amplitude'])
df


In [ ]:
df.describe()

Shift the DOA's to the Left by one so that the Current I and Q data Align with the current DOA while using TimeSeriesGenerator to Window the Data

In [ ]:
# Shift the DOA to the left by one to use the current DOA label with the current I and Q data.
# Normally the TimeseriesGenerator function uses past values to predict the future but we would like current data to help predict the current samples' DOA
print(len(y_train_scale))
print(len(y_test_scale))
y_train_scale=insert(y_train_scale, 0, 0)
y_train_scale=np.delete(y_train_scale, -1)
y_test_scale=insert(y_test_scale, 0,0)
y_test_scale=np.delete(y_test_scale, -1)
print(y_train_scale)
print(len(y_train_scale))
print(y_test_scale)
print(len(y_test_scale))

Create the Windows Neccesary for the LSTM model using the TimeSeriesGenerator Function

In [ ]:
# Create the windows neccesary for the LSTM model within tensorflow keras
# Below are the arguments of the timeseries_dataset_from_array function
# The function takes a numpy array and makes a timeseries out of it
NumSampsPerPing= 4301-24+1
data=x_train_scale # the data to make the windows
targets=y_train_scale # time steps in the data (don't need)
sequence_length=10 # window length
sequence_stride=1# period between successive output sequences
sampling_rate=1 # period between successive individual timesteps     within sequences
batch_size=NumSampsPerPing # number of time series samples in each batch
shuffle=False #shuffle the data before making the windows
seed=None # is related to shuffle
start_index=None # is related to shuffle
end_index=None # is related to shuffle
n_features=21

test_data=x_test_scale
test_targets=y_test_scale

dataRows = data.shape[0]
validationStartRow = dataRows - NumSampsPerPing*50
print(validationStartRow, ',',dataRows)

train_data = data[:validationStartRow-1]
#train_data.shape[0]
train_targets = targets[:validationStartRow-1]
#train_targets.shape[0]
val_data = data[validationStartRow:]
#val_data.shape[0], dataRows-validationStartRow
val_targets = targets[validationStartRow:]

In [ ]:
train_data.shape

In [ ]:
val_data.shape

In [ ]:


#inputs=TimeseriesGenerator(data, targets,sequence_length, batch_size)
train_inputs=TimeseriesGenerator(data=train_data,targets=train_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=batch_size)

val_inputs = TimeseriesGenerator(data=val_data,targets=val_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=batch_size)

#test_inputs=TimeseriesGenerator(test_data[0:40000], -1*test_targets[0:40000],sequence_length, batch_size)
test_inputs=TimeseriesGenerator(data=test_data,targets=test_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=batch_size)

#inputs=tf.keras.preprocessing.timeseries_dataset_from_array(data, targets, sequence_length, sequence_stride, sampling_rate, batch_size, shuffle)
#print(type(inputs))

#print('done')

In [ ]:
xArr = np.ndarray(shape=(900, batch_size, sequence_length, n_features))
yArr = np.ndarray(shape=(900, batch_size))
for i in range(900):    
    [x0, y0]=train_inputs[i]
    xArr[i] = x0
    yArr[i] = y0

In [ ]:
xArr = xArr.reshape((900, 4278, 10, 21, 1))
yArr = yArr.reshape((900, 4278, 1))
x0.shape, y0.shape, type(x0), xArr.shape, yArr.shape

In [ ]:
# print the first couple of samples to see if everything is lining up
for i in range(100):
	x, y = train_inputs[i]
	print('%s => %s' % (x, y))

In [ ]:
# print the first couple of samples to see if everything everything is lining up
for i in range(100):
	x, y = test_inputs[i]
	print('%s => %s' % (x, y))

Define the Model's Structure

In [ ]:
# clear the model!
model = 0
#batch_size = 4278
#sequence_length=10
#n_features=21

batch_size, sequence_length, n_features


In [ ]:
cnn = Sequential(name='MLDOA_CNN')
# https://androidkt.com/filters-kernel-size-input-shape-in-conv2d-layer/
    # acceptable filter size f for something of size [32, 32, 3] is f*f*3 where f= 3, 5, 7 and so on...
    # in our case, we have [batch_size, sequence_length, num_features] => [4278, 10, 21], so #filters = f*f*21, lets start w/ f = 3
    # 3*3*21 = 9*21 = 189

    # [sequence_length, num_features, 1] => [10, 21, 1], so #filters = f*f*1, say f = 7, num filters = 49
kernelSize = 5


cnn.add(Conv2D(filters=189, kernel_size=kernelSize, strides=1, activation='relu', input_shape=(batch_size, sequence_length,n_features)))
#cnn.add(Conv2D(filters=49, kernel_size=kernelSize, strides=1, activation='relu', input_shape=(sequence_length,n_features, 1)))
cnn.add(MaxPooling2D(pool_size=(kernelSize, kernelSize)))
cnn.add(Flatten())


In [ ]:
# With all the preproccessing done, it is time to define the model
# Define the LSTM Model
kernelSize = 2

model= Sequential(name='MLDOASequential')
model.add(TimeDistributed(Conv1D(filters=49, kernel_size=kernelSize, strides=1, activation='relu'), input_shape=(batch_size,sequence_length,n_features), name='TD_Conv1D'))
model.add(TimeDistributed(MaxPooling1D(pool_size=(kernelSize)), name='TD_MaxPooling1D'))
#model.add(TimeDistributed(Conv2D(filters=189, kernel_size=kernelSize, strides=1, activation='relu'), input_shape=(batch_size,sequence_length,n_features, 1), name='TD_Conv2D'))
#model.add(TimeDistributed(MaxPooling2D(pool_size=(kernelSize, kernelSize)), name='TD_MaxPooling2D'))
model.add(TimeDistributed(Flatten(), name='TD_Flatten'))
#model.add(LSTM(units=21, name='LSTM1', activation='tanh', input_shape=(sequence_length,n_features), return_sequences=True))
#model.add(LSTM(units=64, name='LSTM1', activation='tanh', batch_input_shape=(batch_size, sequence_length, n_features),input_shape=(sequence_length,n_features), return_sequences=True, stateful=False, bias_initializer='ones'))
model.add(LSTM(units=64, name='LSTM1', activation='tanh', input_shape=(sequence_length,n_features), return_sequences=True, stateful=False, bias_initializer='ones'))
#model.add(Dense(units=))
model.add(Dense(units=100, name='Dense1', activation='relu'))
model.add(Dropout(0.2, name='Dropout1'))

model.add(LSTM(units=416, name='LSTM2', activation='tanh', return_sequences=True, stateful=False, bias_initializer='ones'))
model.add(Dense(units=100, name='Dense2'))
#model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.2, name='Dropout2'))

model.add(LSTM(units=512, name='LSTM3', activation='tanh', return_sequences=False, stateful=False, bias_initializer='ones'))
model.add(Dense(units=100, name='Dense3'))
model.add(Dropout(0.2, name='Dropout3'))

#model.add(LSTM(units=84, name='LSTM3', activation='tanh', return_sequences=True))
#model.add(Dense(units=100))
#model.add(Dense(units=100, activation = 'linear'))
#model.add(Dropout(0.2))
#model.add(LSTM(units=320, activation='tanh', return_sequences=True))
#model.add(Dropout(0.2))
#model.add(LSTM(units=168, name='LSTM4', activation='tanh'))
#model.add(Dense(units=100))
#model.add(Dense(units=100, activation = 'linear'))
#model.add(Dropout(0.2))
#model.add(LSTM(units=480))
#model.add(Dropout(0.2))


model.add(Dense(units=1, name='DenseOutput')) #one output (namely: DOA)



In [ ]:
model.summary()

Train the Model using the Specified Hyperparmeters

In [ ]:
# epoch learning rate scheduler for 10 epochs.
def EpochScheduler10_2(epoch, lr):
    if epoch == 0:
        return 0.1
    if epoch == 1:
        return 0.1
    elif epoch == 2:
        return 0.1
    elif epoch == 3:
        return 0.05
    elif epoch == 4:
        return 0.05
    elif epoch == 5:
        return 0.05
    elif epoch == 6:
        return 0.025
    elif epoch == 7:
        return 0.025
    elif epoch == 8:
        return 0.025
    elif epoch == 9:
        return 0.01
    elif epoch == 10:
        return 0.01
    elif epoch == 11:
        return 0.01
    else:
        return 0.01


In [ ]:
# epoch learning rate scheduler for 10 epochs.
def EpochScheduler10(epoch, lr):
    if epoch == 0:
        return 0.01
    if epoch == 1:
        return 0.01
    elif epoch == 2:
        return 0.01
    elif epoch == 3:
        return 0.005
    elif epoch == 4:
        return 0.005
    elif epoch == 5:
        return 0.005
    elif epoch == 6:
        return 0.0025
    elif epoch == 7:
        return 0.0025
    elif epoch == 8:
        return 0.0025
    elif epoch == 9:
        return 0.001
    elif epoch == 10:
        return 0.001
    elif epoch == 11:
        return 0.001
    else:
        return 0.001


In [ ]:
#Now it is time to train the model
opt=tf.keras.optimizers.Adam(learning_rate=0.01)#,decay=1e-5)
#opt = tf.keras.optimizers.Adam()
mMSE = tf.keras.metrics.MeanSquaredError()     # metric for Mean Squared Error
mRMSE = tf.keras.metrics.RootMeanSquaredError()

model.compile(loss='mse', optimizer=opt)# metrics=[mMSE, mRMSE])

# Allow for early stopping so that the model does not overfit the training dataset
es= EarlyStopping(monitor='loss', mode='min',verbose=1,patience=10)

# change learning rates per epoch
lrCallback = tf.keras.callbacks.LearningRateScheduler(EpochScheduler10_2)

# Model Checkpoint to save good runs
#cp = ModelCheckpoint('model/', save_best_only=True)
cp = ModelCheckpoint('model/', monitor='loss',save_best_only=True)

t0=time.time()
#history= model.fit(inputs,steps_per_epoch=4000,epochs=200,verbose=1, callbacks=[es])
#history= model.fit(train_inputs, steps_per_epoch=4000, validation_data=val_inputs, epochs=100, callbacks=[cp])
#history= model.fit(train_inputs, epochs=1, batch_size=None,callbacks=[cp, es])
history= model.fit(x=xArr, y=yArr, steps_per_epoch = 30, epochs=100, callbacks=[cp])
t1=time.time()
print("The total run time to train was %.2f seconds"%(t1-t0))

# plot the loss function
plt.figure()
plt.semilogy(history.history['loss'])
plt.xlabel('epoch'); plt.ylabel('loss')


In [ ]:
model = keras.models.load_model('model/')

Using the Model, Predict the DOA's on the Testing Dataset

In [ ]:
#use the model to predict on the testing dataset
#for right now test_inputs has only the 40,001 samples
predicted=model.predict(test_inputs, verbose=1)

In [ ]:
predicted.shape

Put the predicted DOA's back into a csv file format

In [ ]:
#Ping |  Num Samp Num | PORT or STBD (0 or 1) |  TWTT |  Predicted DOA   <=======Output columns.
OutputCSVdf=test.iloc[:,[0,1,2,3]]
#OutputCSVdf= pd.DataFrame(test.iloc[:,[0,1,2,3]], columns=['PingNumber','SampNumber','PortStbd', 'SampleTime'])
zerosArray= np.arange(sequence_length)*0
predictedShifted=np.append(zerosArray, predicted)
Predicteddf= pd.DataFrame(predictedShifted,columns=['PredictedDOA'])
OutputCSVdf=OutputCSVdf.join(Predicteddf)

In [ ]:
OutputCSVdf

In [ ]:
OutputCSVdf.to_csv(path_or_buf="D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PredictedOutputTestMB3.csv")

In [ ]:
test.iloc[:,[0,1,2,3]]

In [ ]:
zerosArray= np.arange(sequence_length)*0
zerosArray

In [ ]:
predictedShifted=np.append(zerosArray, predicted)
predictedShifted

In [ ]:
print(predicted[0:21])

In [ ]:
print(y_test[10:21])

In [ ]:
type(predicted)
predicted.shape

In [ ]:
predicted_reshape= predicted.reshape(len(predicted),1)

In [ ]:
## JUNK 1st 10 SAMPLES ARE JUNK ANYWAYS DON'T USE

plt.figure(facecolor='white')
plt.plot(-y_test[20:30], '.')
plt.plot(predicted[20:30], '.')
plt.title("Actual")
plt.legend(['Actual','Predicted'])
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.show()

In [ ]:
plt.figure(facecolor='white')
plt.plot(y_test[0:40000])
plt.plot(predicted[0:40000])
plt.title("Predicted v Actual")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Actual','Predicted'])
plt.show()



Compare the Actual Testing DOA's to the Predicted DOA's

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)


plt.figure(facecolor='white')
plt.plot(y_test[0:10], '.')
plt.plot(predicted[0:10], '.')
plt.title("Actual")
plt.legend(['Actual','Predicted'])
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.show()

plt.figure(facecolor='white')
plt.plot(y_test[0:40000])
plt.plot(predicted_reshape[0:40000])
plt.title("Actual")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Actual','Predicted'])
plt.show()


plt.figure(facecolor='white')
plt.plot(y_test[0:40000])
plt.title("Actual")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Actual'])
plt.show()


plt.figure(facecolor='white')
plt.plot(predicted_reshape[0:40000], 'orange')
plt.title("Actual")
plt.xlabel('Samples')
plt.ylabel('DOA in degrees')
plt.legend(['Predicted'])
plt.show()


In [ ]:
df2= pd.DataFrame(predicted, columns= ['DOA'])
df2